In [34]:
import infomap
from infomap import Infomap
import pandas as pd
import networkx as nx
import numpy as np 
import matplotlib.pyplot as plt 
import pickle as pk
from scipy import stats
import gseapy as gp


In [35]:
def overlap_jaccard(list1,list2):
    intersction_term= len(set(list1) & set(list2))
    denominator = len(set(list1).union(set(list2)))
    overlap_jaccard_coeff = intersction_term/denominator
    return overlap_jaccard_coeff



def read_gene_annotation(file_path):
    """
    Reads a gene annotation file and stores it as a dictionary.
    
    Parameters:
        file_path (str): Path to the input text file.
        
    Returns:
        dict: Dictionary where keys are annotations and values are lists of genes.
    """
    gene_dict = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')  # Split by tab
            if len(parts) > 1:
                annotation = parts[0].strip()
                genes = [gene.strip() for gene in parts[1:] if gene.strip()]
                gene_dict[annotation] = genes
    
    return gene_dict

def overlap_jaccard(list1,list2):
    intersction_term= len(set(list1) & set(list2))
    denominator = len(set(list1).union(set(list2)))
    overlap_jaccard_coeff = intersction_term/denominator
    return overlap_jaccard_coeff

def overlap_genelists(lstA, lstB, background):
    import scipy.stats as stats
    """
    Accepts to lists
    M is the population size (previously N)
    n is the number of successes in the population
    N is the sample size (previously n)
    x is still the number of drawn “successes”
    """
    setA= set(lstA)
    setB= set(lstB)
    M= background #total number of genes
    n= len(setA)
    N= len(setB)
    x= len(setA.intersection(setB))


    return str(stats.hypergeom.sf(x-1, M, n, N))



#Here we define the adjustment
from statsmodels.sandbox.stats.multicomp import multipletests
def fdr_adjustment(list_of_pvals,alpha):    
    return multipletests(list_of_pvals,alpha=alpha,method='fdr_bh')[1] #the benjamin hochberg method is used


def enrichment_fishers(foreground_list,background_lenght):
    ann_list = []
    pval_list = []
    ji_list = []
    for ann,annotation_list in tot_gene_annotation_dict.items():
        pval_ov = float(overlap_genelists(foreground_list,annotation_list,background_lenght)) 
        ji_overlap = overlap_jaccard(foreground_list,annotation_list)
        ann_list.append(ann)
        pval_list.append(pval_ov)
        ji_list.append(ji_overlap)
    adj_pval = fdr_adjustment(pval_list,0.05)
    results_df = pd.DataFrame()
    results_df['Term'] = ann_list
    results_df['Overlap'] = ji_list
    results_df['P-value'] = pval_list
    results_df['FDR'] = adj_pval
    # Sort results by adjusted p-value
    results_df = results_df.sort_values(by='FDR')
    # Compute combined score (-log10 Adjusted P-value * Overlap)
    results_df['Combined Score'] = -np.log10(results_df['FDR']) * results_df['Overlap']
    results_df_sig = results_df[results_df['FDR']<0.05]
    return results_df_sig
    
library_list = ["GO_Molecular_Function_2025","GO_Biological_Process_2025","GO_Cellular_Component_2025","Reactome_Pathways_2024"]
def enr_ranking_leading_genes(pre_rank_df,library_list):   #This function returns the leading genes
    leading_genes_tot=[]
    for lib in library_list:
        c=0
        while c<10:
            try:
                lib_pre_res = gp.prerank(rnk=pre_rank_df, gene_sets=lib,
                                 processes=4,min_size=3,max_size=5000,outdir=None,   #we impose at least 5 enriched common genes
                                 permutation_num=500) # reduce number to speed up testing
                leading_gene_lib_list=[]
                for gene_str in lib_pre_res.res2d['Lead_genes'].tolist():    #lib_pre_res.res2d['Lead_genes'] if in the cluster
                    leading_gene_lib_list.extend(gene_str.split(";"))
                leading_genes_tot.extend(leading_gene_lib_list)
                if 'Lead_genes' in lib_pre_res.res2d:
                    break
            except (NameError, AttributeError):
                c += 1
    return set(leading_genes_tot)

def read_json_to_dict(file):
    import json
    import pandas as pd
    
    # Load the JSON file (replace with your path)
    with open(file, "r") as f:
        possible_dict = json.load(f)
    
    # Initialize an empty dictionary to store DataFrames for each key
    dataframes_dict = {}
    
    # Loop through each key in the possible_dict
    for key, result in possible_dict.items():
        # Initialize an empty list to store the rows for this DataFrame
        data = []
        
        # Iterate through the list of results for each key
        for entry in result:
            # Split FgRatio and BgRatio
            fg_ratio_split = entry['FgRatio'].split('/')
            bg_ratio_split = entry['BgRatio'].split('/')
            
            # Convert the split ratios to numeric values
            fg_ratio = float(fg_ratio_split[0]) / float(fg_ratio_split[1])
            bg_ratio = float(bg_ratio_split[0]) / float(bg_ratio_split[1])
            
            # Prepare the row data with all required columns
            row = {
                'Description': entry['Description'],
                'FgRatio': fg_ratio,
                'BgRatio': bg_ratio,
                'ES': entry['ES'],
                'pvalue': entry['pvalue'],
                'FDR': entry['FDR'],
                '_row': entry['_row']
            }
            
            # Append the row to the data list
            data.append(row)
        
        # Convert the data list into a DataFrame
        df = pd.DataFrame(data)
        
        # Store the DataFrame in the dictionary with the key as its name
        dataframes_dict[key] = df
    return dataframes_dict


In [6]:
#Let's import the random EEN
random_een_df = pd.read_csv("output/random_een.tsv", sep="\t",index_col=0)
random_een = nx.from_pandas_edgelist(random_een_df, 'Exp A', 'Exp B')

In [8]:
#Let's import the chemical-gene interactions from CTD (downloaded on 5th April 2021)
chem_gene_df = pd.read_csv("input/CTD/CTD_chem_gene_ixns.tsv",delimiter= '\t', skipinitialspace=True)
#Here, we filter for only the interactions that regards the H. Sapiens
chem_homo = chem_gene_df[(chem_gene_df['Organism'] == 'Homo sapiens')]

#This cells create a dictionary where each key is a chemical compound and the correspondent value is a genelist 
chem_gene = {}
for i,v in chem_homo.iterrows():
    try:
        chem_gene[v["ChemicalID"]] |= {v["GeneSymbol"]}
    except KeyError as e:
        chem_gene[v["ChemicalID"]] = set([v["GeneSymbol"]])

#Here, we keep only the exposures which perturb at least one gene
chem_gene_cleaned = {}
tot_gene_list=[]
for k,v in chem_gene.items():
    if len(v)>0:
        chem_gene_cleaned[k]=v
        for gene in v:
            tot_gene_list.append(gene)
    else:
        pass

In [11]:
random_een_weighted = nx.Graph()
for e in random_een.edges():
    ji= overlap_jaccard(chem_gene_cleaned[e[0]],chem_gene_cleaned[e[1]])  
    random_een_weighted.add_edge(*e)
    random_een_weighted[e[0]][e[1]]['weight']=ji
    

In [12]:
#from https://mapequation.github.io/infomap/python/infomap.html#infomap.InfomapIterator
#for some more examples on infomap: 
#https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=bd65c716de009b259e33715093fca7d27e06699b&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6368726973626c6f65636b65722f696e666f6d61702d6269706172746974652f626436356337313664653030396232353965333337313530393366636137643237653036363939622f6578616d706c65732f707974686f6e2f696e666f6d61702d6578616d706c65732e6970796e62&logged_in=false&nwo=chrisbloecker%2Finfomap-bipartite&path=examples%2Fpython%2Finfomap-examples.ipynb&platform=android&repository_id=176271675&repository_type=Repository&version=99


im = Infomap(silent=True)
mapping = im.add_networkx_graph(random_een_weighted, weight='weight')
im.run()
for modules in sorted(im.get_multilevel_modules().values()):
    print(modules)

(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)
(1, 1, 1)


In [13]:
backbone_node_com_dict={}
for node,modules in im.get_multilevel_modules(states=True).items():
    backbone_node_com_dict[mapping[node]]=list(modules)


In [14]:
tot_com_list=[]
for node,com_list in backbone_node_com_dict.items():
    tot_com_list.extend(com_list)
print(len(set(tot_com_list)))  #This is the total number of communities

backbone_com_node_dict={}
for com in set(tot_com_list):
    node_list=[]
    for node,com_list in backbone_node_com_dict.items():
        if com in com_list:
            node_list.append(node)
    backbone_com_node_dict[com]=node_list
        
for com,node_list in backbone_com_node_dict.items():
    print("In %s there are %s exposures" %(com, len(node_list)))

2596
In 1 there are 7493 exposures
In 2 there are 797 exposures
In 3 there are 371 exposures
In 4 there are 194 exposures
In 5 there are 39 exposures
In 6 there are 38 exposures
In 7 there are 26 exposures
In 8 there are 32 exposures
In 9 there are 29 exposures
In 10 there are 30 exposures
In 11 there are 22 exposures
In 12 there are 24 exposures
In 13 there are 29 exposures
In 14 there are 22 exposures
In 15 there are 20 exposures
In 16 there are 25 exposures
In 17 there are 20 exposures
In 18 there are 20 exposures
In 19 there are 19 exposures
In 20 there are 17 exposures
In 21 there are 21 exposures
In 22 there are 15 exposures
In 23 there are 12 exposures
In 24 there are 14 exposures
In 25 there are 19 exposures
In 26 there are 12 exposures
In 27 there are 12 exposures
In 28 there are 35 exposures
In 29 there are 34 exposures
In 30 there are 27 exposures
In 31 there are 32 exposures
In 32 there are 27 exposures
In 33 there are 31 exposures
In 34 there are 22 exposures
In 35 there a

In [15]:
first_iteration_infomap={}
second_iteration_infomap={}
third_iteration_infomap={}
fourth_iteration_infomap={}

for com in set(im.get_modules(depth_level=1).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=1).items():
        if com==com1:
            node_list.append(mapping[node])
    first_iteration_infomap[com]=node_list 
    
    
for com in set(im.get_modules(depth_level=2).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=2).items():
        if com==com1:
            node_list.append(mapping[node])
    second_iteration_infomap[com]=node_list
    
for com in set(im.get_modules(depth_level=3).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=3).items():
        if com==com1:
            node_list.append(mapping[node])
    third_iteration_infomap[com]=node_list
    
for com in set(im.get_modules(depth_level=4).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=4).items():
        if com==com1:
            node_list.append(mapping[node])
    fourth_iteration_infomap[com]=node_list

In [16]:
len(first_iteration_infomap)

2247

In [17]:
len(second_iteration_infomap)

2567

In [18]:
len(third_iteration_infomap)

2596

In [19]:
len(fourth_iteration_infomap)

2596

In [20]:
len(set(im.get_modules(depth_level=3).keys()))

9764

In [21]:
with open('output/Communities/Infomap/random_first_iteration_infomap.pickle', 'wb') as handle:
    pk.dump(first_iteration_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)
    
with open('output/Communities/Infomap/random_second_iteration_infomap.pickle', 'wb') as handle:
    pk.dump(second_iteration_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)
    
with open('output/Communities/Infomap/random_third_iteration_infomap.pickle', 'wb') as handle:
    pk.dump(third_iteration_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)


In [22]:
#These communities are the middle and the broad level related to each community

ee_first_level_infomap={}  #middle to fine
for comcom, explist1 in second_iteration_infomap.items():
    comlist=[]
    for com, explist2 in third_iteration_infomap.items():
        if len(set(explist1)&set(explist2))==len(set(explist2)):
           comlist.append(com)
    ee_first_level_infomap[comcom]=comlist

ee_second_level_infomap={}  #broad to middle
for comcom, explist1 in first_iteration_infomap.items():
    comlist=[]
    for com, explist2 in second_iteration_infomap.items():
        if len(set(explist1)&set(explist2))==len(set(explist2)):
           comlist.append(com)
    ee_second_level_infomap[comcom]=comlist
 

In [27]:
with open('output/Communities/Infomap/ee_ji_first_random_infomap_iteration.pickle', 'wb') as handle:
    pk.dump(ee_first_level_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)
    
with open('output/Communities/Infomap/ee_ji_second_random_infomap_iteration.pickle', 'wb') as handle:
    pk.dump(ee_second_level_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)

In [28]:

with open('output/Communities/Infomap/random_third_iteration_infomap.pickle', 'rb') as handle:
    first_level_infomap = pk.load(handle)

with open('output/Communities/Infomap/random_second_iteration_infomap.pickle', 'rb') as handle:
    second_level_infomap = pk.load(handle)

with open('output/Communities/Infomap/random_first_iteration_infomap.pickle', 'rb') as handle:
    third_level_infomap = pk.load(handle)

with open('output/Communities/Infomap/ee_ji_first_random_infomap_iteration.pickle', 'rb') as handle:
    ee_first_level_infomap = pk.load(handle)

with open('output/Communities/Infomap/ee_ji_second_random_infomap_iteration.pickle', 'rb') as handle:
    ee_second_level_infomap = pk.load(handle)

In [29]:
#Let's change the names of these communities based on their size

#BROADEST==third
third_level_infomap_sort=dict(sorted(third_level_infomap.items(), key=lambda item: len(item[1]), reverse = True))
third_renamed_third_conversion_dict={}
renamed_third_level_infomap_sort={}
c=1
for com,explist in third_level_infomap_sort.items():
    renamed_third_level_infomap_sort[str(c)]=explist
    third_renamed_third_conversion_dict[com]=str(c)
    c=c+1
    
#MIDDLE==second

second_renamed_second_conversion_dict={}
renamed_second_level_infomap_sort={}

for br_com,mid_com in ee_second_level_infomap.items():
    if len(mid_com)==1:
        newcom=third_renamed_third_conversion_dict[br_com]
        renamed_second_level_infomap_sort[newcom]=second_level_infomap[mid_com[0]]
        second_renamed_second_conversion_dict[mid_com[0]]=newcom
    else:
        sorted_list=list(sorted(mid_com, key=lambda item: len(second_level_infomap[item]), reverse = True))
        for i in range(len(sorted_list)):
            q=i+1
            newcom=third_renamed_third_conversion_dict[br_com]+"_"+str(q)
            renamed_second_level_infomap_sort[newcom]=second_level_infomap[sorted_list[i]]
            second_renamed_second_conversion_dict[sorted_list[i]]=newcom
        
#FINEST==first  

first_renamed_first_conversion_dict={}
renamed_first_level_infomap_sort={}

for mid_com,fin_com in ee_first_level_infomap.items():
    if len(fin_com)==1:
        newcom=second_renamed_second_conversion_dict[mid_com]
        renamed_first_level_infomap_sort[newcom]=first_level_infomap[fin_com[0]]
        first_renamed_first_conversion_dict[fin_com[0]]=newcom
    else:
        sorted_list=list(sorted(fin_com, key=lambda item: len(first_level_infomap[item]), reverse = True))
        for i in range(len(sorted_list)):
            q=i+1
            newcom=second_renamed_second_conversion_dict[mid_com]+"_"+str(q)
            renamed_first_level_infomap_sort[newcom]=first_level_infomap[sorted_list[i]]
            first_renamed_first_conversion_dict[sorted_list[i]]=newcom
     

In [30]:
#We are using the new nomenclature
fine_grained_communities_list=[]
middle_communities_list=[]
broad_communities_list=[]

for com,explist in renamed_first_level_infomap_sort.items():
    comlist=com.split("_")
    if len(comlist)==1:
        fine_grained_communities_list.append(com)
    elif len(comlist)==2:
        fine_grained_communities_list.append(com)
        middle_communities_list.append(comlist[0])
    else:
        fine_grained_communities_list.append(com)
        middle_communities_list.append(comlist[0]+"_"+comlist[1])
        broad_communities_list.append(comlist[0])

fine_grained_communities_set=set(fine_grained_communities_list)
middle_communities_set=set(middle_communities_list)
broad_communities_set=set(broad_communities_list)
print(len(broad_communities_set))  
print(len(middle_communities_set))  
print(len(fine_grained_communities_set))  

1
6
2596


In [31]:
fine_grained_communities_dict={}
middle_communities_dict={}
broad_communities_dict={}

for com in fine_grained_communities_set:
    fine_grained_communities_dict[com]=renamed_first_level_infomap_sort[com]
    
for com in middle_communities_set:
    try:
        middle_communities_dict[com]=renamed_second_level_infomap_sort[com]
    except:
        middle_communities_dict[com]=renamed_third_level_infomap_sort[com]

for com in broad_communities_set:
    broad_communities_dict[com]=renamed_third_level_infomap_sort[com]
    
with open('output/Communities/Infomap/random_infomap_fine_grained_communities_dict.pickle', 'wb') as handle:
    pk.dump(fine_grained_communities_dict, handle, protocol=pk.HIGHEST_PROTOCOL)

with open('output/Communities/Infomap/random_infomap_middle_communities_dict.pickle', 'wb') as handle:
    pk.dump(middle_communities_dict, handle, protocol=pk.HIGHEST_PROTOCOL)

with open('output/Communities/Infomap/random_infomap_broad_communities_dict.pickle', 'wb') as handle:
    pk.dump(broad_communities_dict, handle, protocol=pk.HIGHEST_PROTOCOL)

In [36]:
#Let's read the annotations files
MF2023_gene_dict = read_gene_annotation("input/Libraries/GO_Molecular_Function_2023.txt")
CC2023_gene_dict = read_gene_annotation("input/Libraries/GO_Cellular_Component_2023.txt")
BP2023_gene_dict = read_gene_annotation("input/Libraries/GO_Biological_Process_2023.txt")
Reactome2024_gene_dict = read_gene_annotation("input/Libraries/Reactome_Pathways_2024.txt")

In [37]:

tot_gene_annotation_dict = {**MF2023_gene_dict, **CC2023_gene_dict, **BP2023_gene_dict, **Reactome2024_gene_dict}

In [38]:
genetic_background = set([x for xs in list(tot_gene_annotation_dict.values()) for x in xs]).union(set([x for xs in list(chem_gene_cleaned.values()) for x in xs]))


In [39]:
ee_first_infomap_iteration_exposures = fine_grained_communities_dict
ee_second_infomap_iteration_exposures = middle_communities_dict
ee_third_infomap_iteration_exposures = broad_communities_dict


In [40]:

#I consider a pre-rank for making a choice on the gene cutoff
#defining for each comunity a set of genes that is well-representative of each cluster
from collections import Counter
pre_rank_first_infomap_cluster_genelist={}
for com,explist in ee_first_infomap_iteration_exposures.items():
    gene_list=[]
    rank_genelist=[]
    if len(explist)>1:
        G_sub = nx.subgraph(random_een_weighted,explist)   #let's create the subgraph corresponding to the cluster
        for edge in G_sub.edges():
            overlap_geneset=chem_gene_cleaned[edge[0]] & chem_gene_cleaned[edge[1]]
            for gene in overlap_geneset:
                gene_list.append(gene)
        count_dict=Counter(gene_list)
        for gene in set(gene_list):
            rank_genelist.append([gene,count_dict[gene]])
    else:
        for gene in chem_gene_cleaned[explist[0]]:
            rank_genelist.append([gene,1])
    pre_rank_first_infomap_cluster_genelist[com]=rank_genelist

pre_rank_first_infomap_cluster_genelist_df={}
for com,genelist in pre_rank_first_infomap_cluster_genelist.items():
    rank_df=pd.DataFrame(columns=('gene', 'rank'))
    for n in range(len(genelist)):
        rank_df.loc[n] = [genelist[n][0], float(genelist[n][1])]
    rank_df=rank_df.sort_values(by=['rank'],ascending=False)
    pre_rank_first_infomap_cluster_genelist_df[com]=rank_df

pre_rank_second_infomap_cluster_genelist={}
for com,explist in ee_second_infomap_iteration_exposures.items():
    gene_list=[]
    rank_genelist=[]
    if len(explist)>1:
        G_sub = nx.subgraph(random_een_weighted,explist)   #let's create the subgraph corresponding to the cluster
        for edge in G_sub.edges():
            overlap_geneset=chem_gene_cleaned[edge[0]] & chem_gene_cleaned[edge[1]]
            for gene in overlap_geneset:
                gene_list.append(gene)
        count_dict=Counter(gene_list)
        for gene in set(gene_list):
            rank_genelist.append([gene,count_dict[gene]])
    else:
        for gene in chem_gene_cleaned[explist[0]]:
            rank_genelist.append([gene,1])
    pre_rank_second_infomap_cluster_genelist[com]=rank_genelist

pre_rank_second_infomap_cluster_genelist_df={}
for com,genelist in pre_rank_second_infomap_cluster_genelist.items():
    rank_df=pd.DataFrame(columns=('gene', 'rank'))
    for n in range(len(genelist)):
        rank_df.loc[n] = [genelist[n][0], float(genelist[n][1])]
    rank_df=rank_df.sort_values(by=['rank'],ascending=False)
    pre_rank_second_infomap_cluster_genelist_df[com]=rank_df

pre_rank_third_infomap_cluster_genelist={}
for com,explist in ee_third_infomap_iteration_exposures.items():
    gene_list=[]
    rank_genelist=[]
    if len(explist)>1:
        G_sub = nx.subgraph(random_een_weighted,explist)   #let's create the subgraph corresponding to the cluster
        for edge in G_sub.edges():
            overlap_geneset=chem_gene_cleaned[edge[0]] & chem_gene_cleaned[edge[1]]
            for gene in overlap_geneset:
                gene_list.append(gene)
        count_dict=Counter(gene_list)
        for gene in set(gene_list):
            rank_genelist.append([gene,count_dict[gene]])
    else:
        for gene in chem_gene_cleaned[explist[0]]:
            rank_genelist.append([gene,1])
    pre_rank_third_infomap_cluster_genelist[com]=rank_genelist

pre_rank_third_infomap_cluster_genelist_df={}
for com,genelist in pre_rank_third_infomap_cluster_genelist.items():
    rank_df=pd.DataFrame(columns=('gene', 'rank'))
    for n in range(len(genelist)):
        rank_df.loc[n] = [genelist[n][0], float(genelist[n][1])]
    rank_df=rank_df.sort_values(by=['rank'],ascending=False)
    pre_rank_third_infomap_cluster_genelist_df[com]=rank_df


In [ ]:
#Let's associate for each community the most meaningful genes (those communities associated with more than 20 genes, we will use the leading genes of the ranked enrichment by frequency)
#First infomap interation
#Now we will run the traditional enrichment analysis for (Fisher's exact test + BH correction for those communties
#that perturb less than 20 genes, otherwise we will collect the leading genes from the GSEA with ranking
#and then performing again a traditional enrichment analysis

first_infomap_cluster_genes = {}
second_infomap_cluster_genes = {}
third_infomap_cluster_genes = {}

for com in list(pre_rank_first_infomap_cluster_genelist_df.keys()):
    num_of_genes=len(pre_rank_first_infomap_cluster_genelist_df[com])
    if num_of_genes>20:
        try:
            lead_genelist = list(enr_ranking_leading_genes(pre_rank_first_infomap_cluster_genelist_df[com],library_list))
            first_infomap_cluster_genes[com] = lead_genelist
        except:
            first_infomap_cluster_genes[com] = list(pre_rank_first_infomap_cluster_genelist_df[com]["gene"])
    else:
        first_infomap_cluster_genes[com] = list(pre_rank_first_infomap_cluster_genelist_df[com]["gene"])

#Second infomap interation
for com in list(pre_rank_second_infomap_cluster_genelist_df.keys()):
    num_of_genes=len(pre_rank_second_infomap_cluster_genelist_df[com])
    if num_of_genes>20:
        try:
            lead_genelist = list(enr_ranking_leading_genes(pre_rank_second_infomap_cluster_genelist_df[com],library_list))
            second_infomap_cluster_genes[com] = lead_genelist
        except:
            second_infomap_cluster_genes[com] = list(pre_rank_second_infomap_cluster_genelist_df[com]["gene"])
    else:
        second_infomap_cluster_genes[com] = list(pre_rank_second_infomap_cluster_genelist_df[com]["gene"])
        
#Third infomap interation
for com in list(pre_rank_third_infomap_cluster_genelist_df.keys()):
    num_of_genes=len(pre_rank_third_infomap_cluster_genelist_df[com])
    if num_of_genes>20:
        try:
            lead_genelist = list(enr_ranking_leading_genes(pre_rank_third_infomap_cluster_genelist_df[com],library_list))
            third_infomap_cluster_genes[com] = lead_genelist
        except:
            third_infomap_cluster_genes[com] = list(pre_rank_third_infomap_cluster_genelist_df[com]["gene"])
    else:
        third_infomap_cluster_genes[com] = list(pre_rank_third_infomap_cluster_genelist_df[com]["gene"])


C:\Users\salvo\AppData\Local\Temp\ipykernel_5844\1453190568.py:94: DeprecationWarning: processes is deprecated; use threads
  lib_pre_res = gp.prerank(rnk=pre_rank_df, gene_sets=lib,
2025-04-16 11:27:57,469 [WARNING] Duplicated values found in preranked stats: 99.65% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
C:\Users\salvo\AppData\Local\Temp\ipykernel_5844\1453190568.py:94: DeprecationWarning: processes is deprecated; use threads
  lib_pre_res = gp.prerank(rnk=pre_rank_df, gene_sets=lib,
2025-04-16 11:27:57,669 [WARNING] Duplicated values found in preranked stats: 99.65% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
C:\Users\salvo\AppData\Local\Temp\ipykernel_5844\1453190568.py:94: DeprecationWarning: processes is deprecated; use threads
  lib_pre_res = gp.prerank(rnk=pre_rank_df, gene_sets=lib,
2025-04-16 11:27:58,099 [WARNING] Duplicated values found in preranked stats: 99.65% of genes
The o

In [42]:
import json
# Save the JSON data to a file
with open("output/Communities/Infomap/random_first_infomap_cluster_genes.json", "w") as f:
    json.dump(first_infomap_cluster_genes, f)
with open("output/Communities/Infomap/random_second_infomap_cluster_genes.json", "w") as f:
    json.dump(second_infomap_cluster_genes, f)
with open("output/Communities/Infomap/random_third_infomap_cluster_genes.json", "w") as f:
    json.dump(third_infomap_cluster_genes, f)

In [ ]:
#The enrichment of these genes is performed in the R notebook Random_Infomap_ORA, and the comparison with the EEN communities
#is performed in the Python notebook Random Network Comparison


Gene Part
===

In [ ]:
#Let's import the randomized EEN
random_ggn_df = pd.read_csv("output/random_ggn.tsv", sep="\t",index_col=0)
random_ggn = nx.from_pandas_edgelist(random_ggn_df, 'Gene A', 'Gene B')

ggn = nx.read_weighted_edgelist("output/backbone_gene_graph_significant_weighted.edgelist")

In [ ]:
def overlap_jaccard(list1,list2):
    intersction_term= len(set(list1) & set(list2))
    denominator = len(set(list1).union(set(list2)))
    overlap_jaccard_coeff = intersction_term/denominator
    return overlap_jaccard_coeff

In [ ]:
#Let's try to associate some exposures or some chemical class to these communities
gene_chem = {}
for i,v in chem_homo.iterrows():
    try:
        gene_chem[v["GeneSymbol"]] |= {v["ChemicalID"]}
    except KeyError as e:
        gene_chem[v["GeneSymbol"]] = set([v["ChemicalID"]])
        
gene_chem_cleaned={}
for k,v in gene_chem.items():
    if len(v)!=0:
        gene_chem_cleaned[k]=v
    else:
        pass
#Here, we define the fisher test that we will use
def overlap_set_genes(setA, setB,background):    
    """
    Accepts to lists
    M is the population size (previously N)
    n is the number of successes in the population
    N is the sample size (previously n)
    x is still the number of drawn “successes”
    """
    M= len(background) #total number of exposures in chem_gene_cleaned
    n= len(setA)
    N= len(setB)
    x= len(setA.intersection(setB))
    return str(stats.hypergeom.sf(x-1, M, n, N))


from statsmodels.sandbox.stats.multicomp import multipletests
def fdr_adjustment(list_of_pvals,alpha):    
    return multipletests(list_of_pvals,alpha=alpha,method='fdr_bh')[1] #the benjamin hochberg method is used


In [ ]:

random_ggn_weighted = nx.Graph()
for e in random_ggn.edges():
    ji= overlap_jaccard(gene_chem_cleaned[e[0]],gene_chem_cleaned[e[1]])  
    random_ggn_weighted.add_edge(*e)
    random_ggn_weighted[e[0]][e[1]]['weight']=ji
  

In [ ]:
im = Infomap(silent=True)
mapping = im.add_networkx_graph(random_ggn_weighted, weight='weight')
im.run()
for modules in sorted(im.get_multilevel_modules().values()):
    print(modules)

In [ ]:
backbone_node_com_dict={}
for node,modules in im.get_multilevel_modules(states=True).items():
    backbone_node_com_dict[mapping[node]]=list(modules)

In [ ]:
tot_com_list=[]
for node,com_list in backbone_node_com_dict.items():
    tot_com_list.extend(com_list)
print(len(set(tot_com_list)))  #This is the total number of communities

backbone_com_node_dict={}
for com in set(tot_com_list):
    node_list=[]
    for node,com_list in backbone_node_com_dict.items():
        if com in com_list:
            node_list.append(node)
    backbone_com_node_dict[com]=node_list
        
for com,node_list in backbone_com_node_dict.items():
    print("In %s there are %s genes" %(com, len(node_list)))

In [ ]:
first_iteration_infomap={}
second_iteration_infomap={}
third_iteration_infomap={}
fourth_iteration_infomap={}

for com in set(im.get_modules(depth_level=1).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=1).items():
        if com==com1:
            node_list.append(mapping[node])
    first_iteration_infomap[com]=node_list 
    
    
for com in set(im.get_modules(depth_level=2).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=2).items():
        if com==com1:
            node_list.append(mapping[node])
    second_iteration_infomap[com]=node_list
    
for com in set(im.get_modules(depth_level=3).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=3).items():
        if com==com1:
            node_list.append(mapping[node])
    third_iteration_infomap[com]=node_list
    
for com in set(im.get_modules(depth_level=4).values()):
    node_list=[]
    for node,com1 in im.get_modules(depth_level=4).items():
        if com==com1:
            node_list.append(mapping[node])
    fourth_iteration_infomap[com]=node_list

In [ ]:
with open('output/Communities/Infomap/random_ggn_first_iteration_infomap.pickle', 'wb') as handle:
    pk.dump(first_iteration_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)
    

In [ ]:
#These communities are the middle and the broad level related to each community

gg_first_level_infomap={}  #middle to fine
for comcom, explist1 in second_iteration_infomap.items():
    comlist=[]
    for com, explist2 in third_iteration_infomap.items():
        if len(set(explist1)&set(explist2))==len(set(explist2)):
           comlist.append(com)
    gg_first_level_infomap[comcom]=comlist


In [ ]:
with open('output/Communities/Infomap/gg_ji_first_random_infomap_iteration.pickle', 'wb') as handle:
    pk.dump(gg_first_level_infomap, handle, protocol=pk.HIGHEST_PROTOCOL)


In [ ]:
with open('Communities/Infomap/random_ggn_third_iteration_infomap.pickle', 'rb') as handle:
    first_level_infomap = pk.load(handle)
with open('Communities/Infomap/gg_ji_first_random_infomap_iteration.pickle', 'rb') as handle:
    ee_first_level_infomap = pk.load(handle)


In [ ]:
#Let's change the names of these communities based on their size
    
#FINEST==first  

first_renamed_first_conversion_dict={}
renamed_first_level_infomap_sort={}

for mid_com,fin_com in ee_first_level_infomap.items():
    if len(fin_com)==1:
        newcom=second_renamed_second_conversion_dict[mid_com]
        renamed_first_level_infomap_sort[newcom]=first_level_infomap[fin_com[0]]
        first_renamed_first_conversion_dict[fin_com[0]]=newcom
    else:
        sorted_list=list(sorted(fin_com, key=lambda item: len(first_level_infomap[item]), reverse = True))
        for i in range(len(sorted_list)):
            q=i+1
            newcom=second_renamed_second_conversion_dict[mid_com]+"_"+str(q)
            renamed_first_level_infomap_sort[newcom]=first_level_infomap[sorted_list[i]]
            first_renamed_first_conversion_dict[sorted_list[i]]=newcom
     

In [ ]:
#We are using the new nomenclature
fine_grained_communities_list=[]

for com,explist in renamed_first_level_infomap_sort.items():
    comlist=com.split("_")
    if len(comlist)==1:
        fine_grained_communities_list.append(com)

fine_grained_communities_set=set(fine_grained_communities_list)

print(len(fine_grained_communities_set))  


In [ ]:
fine_grained_communities_dict={}

for com in fine_grained_communities_set:
    fine_grained_communities_dict[com]=renamed_first_level_infomap_sort[com]
 

In [ ]:
with open('output/Communities/Infomap/random_ggn_infomap_fine_grained_communities_dict.pickle', 'wb') as handle:
    pk.dump(fine_grained_communities_dict, handle, protocol=pk.HIGHEST_PROTOCOL)


In [ ]:
#Enrichments and comparisons are calculated in the Python Notebook Network Randomization GGN